# Neural Network Volatility Prediction

This notebook demonstrates neural network-driven volatility prediction using LSTM, Transformer, and reinforcement learning, with comparison to traditional econometric models.

## 1. Import Libraries and Set Up Environment

Import Python libraries for data manipulation, deep learning (TensorFlow/Keras, PyTorch), and visualization. Set random seeds for reproducibility.

In [ ]:
# Data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning frameworks
import tensorflow as tf
from tensorflow import keras
import torch
import torch.nn as nn

# Econometric models
from arch import arch_model

# Miscellaneous
import random
import os

# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

## 2. Load and Preprocess Financial Time-Series Data

Load historical asset price and volatility data. Handle missing values, normalize features, and split into training, validation, and test sets.

In [ ]:
# Example: Load sample data (replace with your own data source)
# For demonstration, we'll use Yahoo Finance data via yfinance
import yfinance as yf

symbol = 'SPY'
data = yf.download(symbol, start='2010-01-01', end='2023-01-01')
data = data[['Adj Close']].rename(columns={'Adj Close': 'price'})

# Calculate daily returns
data['return'] = np.log(data['price'] / data['price'].shift(1))
data['volatility'] = data['return'].rolling(window=21).std() * np.sqrt(252)

# Drop missing values
data = data.dropna()

# Normalize features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[['price', 'return', 'volatility']] = scaler.fit_transform(data[['price', 'return', 'volatility']])

# Split into train, validation, and test sets
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train = data.iloc[:train_size]
val = data.iloc[train_size:train_size+val_size]
test = data.iloc[train_size+val_size:]

## 3. Feature Engineering for Volatility Prediction

Generate features such as log returns, rolling statistics, and lagged volatility measures to enhance model input.

In [ ]:
# Feature engineering
def add_features(df):
    df = df.copy()
    df['return_lag1'] = df['return'].shift(1)
    df['return_lag2'] = df['return'].shift(2)
    df['volatility_lag1'] = df['volatility'].shift(1)
    df['volatility_lag2'] = df['volatility'].shift(2)
    df['rolling_mean_5'] = df['return'].rolling(window=5).mean()
    df['rolling_std_5'] = df['return'].rolling(window=5).std()
    df['rolling_mean_21'] = df['return'].rolling(window=21).mean()
    df['rolling_std_21'] = df['return'].rolling(window=21).std()
    return df

train = add_features(train).dropna()
val = add_features(val).dropna()
test = add_features(test).dropna()

feature_cols = [
    'price', 'return', 'return_lag1', 'return_lag2',
    'volatility_lag1', 'volatility_lag2',
    'rolling_mean_5', 'rolling_std_5',
    'rolling_mean_21', 'rolling_std_21'
]
target_col = 'volatility'

## 4. Build and Train LSTM Model

Define, compile, and train an LSTM neural network for volatility forecasting. Track training and validation loss.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Prepare data for LSTM
def create_sequences(X, y, seq_length=21):
    Xs, ys = [], []
    for i in range(len(X) - seq_length):
        Xs.append(X[i:(i+seq_length)])
        ys.append(y[i+seq_length])
    return np.array(Xs), np.array(ys)

seq_length = 21
X_train, y_train = create_sequences(train[feature_cols].values, train[target_col].values, seq_length)
X_val, y_val = create_sequences(val[feature_cols].values, val[target_col].values, seq_length)
X_test, y_test = create_sequences(test[feature_cols].values, test[target_col].values, seq_length)

# Build LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(seq_length, len(feature_cols)), return_sequences=False),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')
history = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    verbose=1
)

## 5. Build and Train Transformer Model

Implement a Transformer-based model for time-series volatility prediction. Train and evaluate its performance.

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Model

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = Dense(ff_dim, activation="relu")(res)
    x = Dropout(dropout)(x)
    x = LayerNormalization(epsilon=1e-6)(x)
    return x + res

input_layer = Input(shape=(seq_length, len(feature_cols)))
x = transformer_encoder(input_layer, head_size=32, num_heads=2, ff_dim=64, dropout=0.1)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(32, activation="relu")(x)
output_layer = Dense(1)(x)

transformer_model = Model(inputs=input_layer, outputs=output_layer)
transformer_model.compile(optimizer='adam', loss='mse')

history_transformer = transformer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    verbose=1
)

## 6. Monte Carlo-Based Reinforcement Learning for Trade Optimization

Apply Monte Carlo simulations and reinforcement learning (e.g., DQN or policy gradient) to optimize trading strategies based on predicted volatility.

In [ ]:
# Example: Simple Monte Carlo simulation for trading strategy based on predicted volatility

# For demonstration, use LSTM predictions on test set
lstm_preds = lstm_model.predict(X_test).flatten()

# Simulate a simple volatility-based trading strategy
# If predicted volatility is above median, reduce position; else, increase position
positions = np.where(lstm_preds > np.median(lstm_preds), 0.5, 1.5)  # leverage factor

returns = test[target_col].values[seq_length:] * positions  # pseudo returns
cumulative_returns = np.cumprod(1 + returns) - 1

plt.figure(figsize=(10, 4))
plt.plot(cumulative_returns, label='Volatility-Adjusted Strategy')
plt.title('Monte Carlo Simulated Trading Strategy')
plt.xlabel('Time')
plt.ylabel('Cumulative Return')
plt.legend()
plt.show()

# For full RL, consider using stable-baselines3 or keras-rl for DQN/Policy Gradient

## 7. Implement Traditional Econometric Volatility Models

Fit and forecast volatility using models such as GARCH or EWMA for baseline comparison.

In [ ]:
# Fit GARCH(1,1) model as a traditional baseline
garch_model = arch_model(train['return'], vol='Garch', p=1, q=1)
garch_fitted = garch_model.fit(disp='off')

# Forecast volatility on test set
garch_forecast = garch_fitted.forecast(horizon=len(test), start=train.index[-1])
garch_vol = np.sqrt(garch_forecast.variance.values[-1, :])

plt.figure(figsize=(10, 4))
plt.plot(test.index[seq_length:], y_test, label='True Volatility')
plt.plot(test.index[seq_length:], garch_vol[seq_length:], label='GARCH Forecast')
plt.title('GARCH Volatility Forecast vs True')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.show()

## 8. Compare Model Performance and Visualize Results

Evaluate and compare the predictive accuracy and trading outcomes of all models. Visualize results with plots and summary statistics.

In [ ]:
from sklearn.metrics import mean_squared_error

# LSTM predictions
lstm_preds = lstm_model.predict(X_test).flatten()
# Transformer predictions
transformer_preds = transformer_model.predict(X_test).flatten()
# GARCH predictions already computed

# Compute MSE
mse_lstm = mean_squared_error(y_test, lstm_preds)
mse_transformer = mean_squared_error(y_test, transformer_preds)
mse_garch = mean_squared_error(y_test, garch_vol[seq_length:])

print(f"LSTM MSE: {mse_lstm:.4f}")
print(f"Transformer MSE: {mse_transformer:.4f}")
print(f"GARCH MSE: {mse_garch:.4f}")

# Plot comparison
plt.figure(figsize=(12, 6))
plt.plot(test.index[seq_length:], y_test, label='True Volatility', color='black')
plt.plot(test.index[seq_length:], lstm_preds, label='LSTM', alpha=0.7)
plt.plot(test.index[seq_length:], transformer_preds, label='Transformer', alpha=0.7)
plt.plot(test.index[seq_length:], garch_vol[seq_length:], label='GARCH', alpha=0.7)
plt.title('Volatility Prediction: True vs Models')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.show()

---
**Conclusion:**  
This notebook demonstrated the use of LSTM and Transformer neural networks, Monte Carlo-based reinforcement learning, and traditional econometric models for volatility prediction and trading strategy optimization. The results highlight the strengths and weaknesses of each approach for financial risk assessment.